# 2-4 extract 1gram from smali code
- step1 smali 코드 수집
- step2 smali 코드에서 4gram을 추출하여 전체 리스트 및 갯수 확인
- step3 smali 코드에서 비번하게 나타나는 10000개의 리스트를 가지고 malware, benign의 비율 확인
- step4 10000개의 리스트 중에서 영향도 있는 리스트만 각출
- step5 각 apk 파일들의 4gram 수집 및 정리


## step1 smali 코드 수집

In [4]:
import os

dir_names = ["Challenge_andro_2nd_dataset"]
dir_list = dict()

i = 0
print( "start")
for dir_name in dir_names:
    dir_name = dir_name + "_apktools"
    if dir_name.startswith('Benign'):
        tag = 'Benign'
    else :
        tag = 'Malware'
        
    for root, dirs, files in os.walk(dir_name):
        for f in files:
            if f.endswith(".smali"):
                dir_list[root + '/' + f ] = tag
            i+=1
            if i%10000 == 0:
                print( "{}                  \r".format(i), end ='')
                    
print('len', len(dir_list))

start
len 942531               


In [5]:
#dir_list.keys()[:10]
a = dir_list.keys()
a = list(a)
a[:10]

['Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/e.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/c.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/a.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/f.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/b.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/d.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/h.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/g.

## step2 smali 코드에서 1gram을 추출하여 전체 리스트 및 갯수 확인

In [17]:
import pandas as pd

df = pd.read_csv("apk_word_df.csv")
smali1_list = list(df['Unnamed: 0'])

smali1_list[:10]

0            !
1            %
2           %d
3           %s
4    %s&client
5       %s&nid
6          %s\
7            &
8         &amp
9        &nbsp
Name: Unnamed: 0, dtype: object

## step5 각 apk 파일들의 1gram 수집 및 정리

apk_word_list

파일로부터 데이터를 불러와서 해시 값에 따라 counter 값 추가

In [29]:
import nltk
from multiprocessing import Pool
from collections import Counter
import numpy as np

apk_word_list = dict()

def file2strings(key):
    special_char = ['$',',','.',';','-','=','<','>',
                    '1','2','3','4','5','6','7','8','9','0',
                    '{','}','(',')',':','`', '#', "'",'/','"']
    
    with open(key) as f:
        data = f.read()#.encode('ascii', 'replace')
        #data = data.decode('utf-8', 'ignore')
        data = data.lower()
        
        for char in special_char:
            data = data.replace(char,' ')
        
        data = data.split()
        #fourgrams = nltk.ngrams(data, 4)
                
        word_count = Counter()
        filename = key.split("/")[1] # 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/ActivityWMenu$FbLoginDialogListener.smali'
        
        for word in list(data):
            #if word in smali1_list:
                word_count[str(word)] += 1
                
        return filename, dict(word_count.most_common())

with Pool(14) as p:
    print ("start")
    i = 0
    count = len(dir_list)
    
    for filename, word_count in p.imap(file2strings, list(dir_list.keys())):
        apk_word_list[filename] = word_count
        i += 1    
        if i %100 == 0:
            print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')
     
    print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')

start


In [53]:
df = pd.DataFrame(apk_word_list, index=list(smali1_list)).T
df = df.fillna(0)
df

,!,%,%d,%s,%s&client,%s&nid,%s\,&,&amp,&nbsp,...,zone,zoomstate,zoosware,zt,zuikong,zveryatki,zxing,zz,|,~a
000ef8ee5e09458686ca98ac8914eafdc0e5e937ae2bcac90ce591a4ad22d023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001652ebc24cc76dbe3fb27d80eb137fa8b60832958ace7617c6d0c6fdb6d18a,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00d56ee88d368ea75997bce26cd5c90a2d0be8981d355c82acf053768bad43f0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00d7f6f13d72210c232707853f28ca2f0267ccab605e7366c92e336faf6bf6bc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00e5635e0af8a17aca2e22f3fcbd52314c0aa16012ec7af30a31a42aea1dcfe8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00f130be2d96cb994c0b7465d6392b30675a8a1186688ef053e1a0c0b0be25dc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
012bfafab8a84bdeb62a22af97ccafaec924cef3329680787dad16dbb2992ca1,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0160c026ef59c9c0fe74cfb26082858ce4def298114ef904ad50b8d56a7f8b50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01718bfe393c5235cdb559eafe1bc3921238e063c087881fed2fe3db187c4b5e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0178c9d429db092f01bfd5c920be9b2f18791d3b69b4a12eaffdd6e59094bff0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
df.T.sum()

000ef8ee5e09458686ca98ac8914eafdc0e5e937ae2bcac90ce591a4ad22d023      216.0
001652ebc24cc76dbe3fb27d80eb137fa8b60832958ace7617c6d0c6fdb6d18a     3979.0
00d56ee88d368ea75997bce26cd5c90a2d0be8981d355c82acf053768bad43f0      120.0
00d7f6f13d72210c232707853f28ca2f0267ccab605e7366c92e336faf6bf6bc      222.0
00e5635e0af8a17aca2e22f3fcbd52314c0aa16012ec7af30a31a42aea1dcfe8       41.0
00f130be2d96cb994c0b7465d6392b30675a8a1186688ef053e1a0c0b0be25dc      112.0
012bfafab8a84bdeb62a22af97ccafaec924cef3329680787dad16dbb2992ca1      436.0
0160c026ef59c9c0fe74cfb26082858ce4def298114ef904ad50b8d56a7f8b50      876.0
01718bfe393c5235cdb559eafe1bc3921238e063c087881fed2fe3db187c4b5e     5364.0
0178c9d429db092f01bfd5c920be9b2f18791d3b69b4a12eaffdd6e59094bff0      112.0
01a3420d3b50210dd401164ea00a6598348b1b51e64f314e9afe2d15c347db4d     1384.0
01c21bd5f65c682f84c036488b30dca7a4ccbf5d34bf12b151d650373f8adce4      179.0
01ddd3d15fe5bf13507d39694f572ee204e9bae82fa3607c780ea7e24491ab26      120.0
01f23cfb8a85

In [55]:
df.sum()

!                   11.0
%                  110.0
%d                  34.0
%s                 231.0
%s&client           70.0
%s&nid              60.0
%s\                  9.0
&                   17.0
&amp                 1.0
&nbsp                0.0
*                  218.0
*\u                  0.0
+                   99.0
?                   23.0
??                   0.0
@                   10.0
[                   52.0
[[b                  0.0
[[f                133.0
[[i                141.0
[[lcom               0.0
[[ljava             21.0
[b                1954.0
[b[b                 3.0
[b[bii              10.0
[bb                  0.0
[bi                 18.0
[bi[bii              8.0
[bii               168.0
[bii[bii            12.0
                   ...  
za                   1.0
zedtema              0.0
zeec                 0.0
zhainanzhi           0.0
zhao                 0.0
zhuaz              150.0
zi                   3.0
ziiii                2.0
ziljava              0.0


In [57]:
df.to_csv("2nd_smali_1gram.csv")